In [1]:
from pathlib import Path
from label_studio_sdk.client import LabelStudio
from fishsense_api_sdk.client import Client
from label_studio_sdk import LseUserApi
from fishsense_api_sdk.models.user import User
import asyncio

In [2]:
URL = "http://localhost:8000"

In [3]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [4]:
fs = Client(URL)
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [5]:
users = list(ls.users.list())

len(users), users[0]

(136,
 LseUserApi(active_organization=1, active_organization_meta={'title': 'Label Studio', 'email': 'e4e@ucsd.edu'}, allow_newsletters=True, avatar=None, custom_hotkeys={}, date_joined=datetime.datetime(2024, 12, 1, 19, 40, 16, 970485, tzinfo=datetime.timezone.utc), email='francisrumsky@gmail.com', first_name='', id=23, initials='fr', last_activity=datetime.datetime(2025, 9, 26, 7, 40, 49, 782060, tzinfo=datetime.timezone.utc), last_name='', org_membership=None, organization_membership=None, phone='', username='francisrumsky'))

In [6]:
fs_users = await asyncio.gather(*(fs.users.get(email=user.email) for user in users))
fs_users = [user for user in fs_users if user is not None]

len(fs_users), fs_users[0]

(119,
 User(id=1, label_studio_id=23, email='francisrumsky@gmail.com', first_name='', last_name='', last_activity=datetime.datetime(2025, 8, 29, 10, 39, 39, 30479, tzinfo=TzInfo(0)), date_joined=datetime.datetime(2024, 12, 1, 19, 40, 16, 970485, tzinfo=TzInfo(0))))

In [7]:
def from_label_studio(user: LseUserApi) -> User:
    """Create a User instance from a Label Studio user."""

    return User(
        id = None,
        label_studio_id=user.id,
        email=user.email,
        first_name=user.first_name,
        last_name=user.last_name,
        last_activity=user.last_activity,
        date_joined=user.date_joined,
    )

In [17]:
for user in users:
    fs_user = await fs.users.get(email=user.email)
    if fs_user is None:
        await fs.users.post(from_label_studio(user))
    else:
        new_fs_user = from_label_studio(user)
        new_fs_user.id = fs_user.id

        await fs.users.put(new_fs_user)